In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("GlobalLandTemperaturesByCountry.csv")
df.head()

dt  AverageTemperature  AverageTemperatureUncertainty Country
0  1743-11-01               4.384                          2.294   Åland
1  1743-12-01                 NaN                            NaN   Åland
2  1744-01-01                 NaN                            NaN   Åland
3  1744-02-01                 NaN                            NaN   Åland
4  1744-03-01                 NaN                            NaN   Åland

In [3]:
df=df.rename(columns={'dt':'date','AverageTemperature':'AvgTemp'})
df.head()

date  AvgTemp  AverageTemperatureUncertainty Country
0  1743-11-01    4.384                          2.294   Åland
1  1743-12-01      NaN                            NaN   Åland
2  1744-01-01      NaN                            NaN   Åland
3  1744-02-01      NaN                            NaN   Åland
4  1744-03-01      NaN                            NaN   Åland

In [4]:
df.isnull().sum()

date                                 0
AvgTemp                          32651
AverageTemperatureUncertainty    31912
Country                              0
dtype: int64

In [5]:
df=df.drop('AverageTemperatureUncertainty',axis=1)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577462 entries, 0 to 577461
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   date     577462 non-null  object 
 1   AvgTemp  544811 non-null  float64
 2   Country  577462 non-null  object 
dtypes: float64(1), object(2)
memory usage: 13.2+ MB


In [7]:
df.describe()

AvgTemp
count  544811.000000
mean       17.193354
std        10.953966
min       -37.658000
25%        10.025000
50%        20.901000
75%        25.814000
max        38.842000

In [8]:
df=df.dropna()#Drops the null rows

In [9]:
df.head(15)

date  AvgTemp Country
0   1743-11-01    4.384   Åland
5   1744-04-01    1.530   Åland
6   1744-05-01    6.702   Åland
7   1744-06-01   11.609   Åland
8   1744-07-01   15.342   Åland
10  1744-09-01   11.702   Åland
11  1744-10-01    5.477   Åland
12  1744-11-01    3.407   Åland
13  1744-12-01   -2.181   Åland
14  1745-01-01   -3.850   Åland
15  1745-02-01   -6.575   Åland
16  1745-03-01   -4.195   Åland
17  1745-04-01   -0.966   Åland
74  1750-01-01    1.091   Åland
75  1750-02-01    0.809   Åland

In [10]:
df_countries=df.groupby(['Country',"date"]).sum().reset_index().sort_values('date',ascending= False)
df_countries.head(15)

Country        date  AvgTemp
318859                     Mexico  2013-09-01   24.990
396261                Puerto Rico  2013-09-01   28.048
89235                      Canada  2013-09-01    7.922
250579                    Jamaica  2013-09-01   28.398
117566                       Cuba  2013-09-01   28.424
418707  Saint Pierre And Miquelon  2013-09-01   13.745
507893   Turks And Caicas Islands  2013-09-01   29.770
144806                El Salvador  2013-09-01   26.405
73944      British Virgin Islands  2013-09-01   28.991
39448                     Bahamas  2013-09-01   28.657
93396              Cayman Islands  2013-09-01   29.753
204940                  Guatemala  2013-09-01   25.655
138770         Dominican Republic  2013-09-01   28.046
534044             Virgin Islands  2013-09-01   28.991
353755                  Nicaragua  2013-09-01   27.664

In [11]:
bounds=(df_countries['date']>'2000-01-01')&(df_countries['date']<'2002-01-01')
df_countries=df_countries.loc[bounds]
df_countries.head()

Country        date  AvgTemp
224270         Hungary  2001-12-01   -4.632
367785          Norway  2001-12-01   -6.983
523345   United States  2001-12-01   -1.060
125344  Czech Republic  2001-12-01   -3.192
209838          Guinea  2001-12-01   25.479

In [23]:
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs,init_notebook_mode, plot, iplot

In [19]:
fig = go.Figure(data=go.Choropleth( 
    locations = df_countries['Country'], 
    locationmode = 'country names',
    z = df_countries['AvgTemp'],
    colorscale = 'emrld', 
    marker_line_color = 'black', marker_line_width = 0.5, ))
fig.update_layout( title_text = 'Climate Change', 
                  title_x = 0.5, 
                  geo=dict( showframe = True, showcoastlines = False, projection_type = 'equirectangular' ) ) 
fig.show()

In [20]:
fig.write_html('img using choropleth.html',auto_open=False)

In [21]:
fig=px.scatter_geo(df_countries,locations=df_countries['Country'],locationmode='country names',
                  hover_name='Country',color='AvgTemp',
                  color_continuous_scale='reds',
                  projection='equirectangular',
                 animation_frame='date')
fig.update_layout( title_text = 'Average Temperature Change', title_x = 0.5,
                  geo=dict( showframe = False, showcoastlines = False )) 

fig.show()

In [24]:
plotly.offline.plot(fig, filename='temp_for_two_years_using_plotly.express.html')

'temp_for_two_years_using_plotly.express.html'

In [25]:
# Manipulating the original dataframe 
df_countrydate = df_countries.groupby(['date','Country']). sum().reset_index() 
#Creating the visualization 
fig = px.choropleth(df_countrydate, locations="Country", locationmode = "country names", 
                    color="AvgTemp", hover_name="Country",
                    animation_frame="date" ) 
fig.update_layout( title_text = 'Average Temperature Change', title_x = 0.5,
                  geo=dict( showframe = False, showcoastlines = False, )) 
fig.show()

In [26]:
plotly.offline.plot(fig, filename='different_visualization_using_choropleth.html')

'different_visualization_using_choropleth.html'

In [27]:
df1=df_countries.loc[df_countries['Country']=='India']
df2=df_countries.loc[df_countries['Country']=='Russia']
df3=df_countries.loc[df_countries['Country']=='United States']
df4=df_countries.loc[df_countries['Country']=='China']
df5=df_countries.loc[df_countries['Country']=='Mexico']
df6=df_countries.loc[df_countries['Country']=='Brazil']
df7=df_countries.loc[df_countries['Country']=='Argentina']
df_some_countries=pd.concat([df1,df2,df3,df4,df5,df6,df7],axis=0,ignore_index=True)
df_some_countries.tail(15)

Country        date  AvgTemp
146  Argentina  2001-04-01   14.678
147  Argentina  2001-03-01   19.219
148  Argentina  2001-02-01   22.131
149  Argentina  2001-01-01   21.770
150  Argentina  2000-12-01   20.472
151  Argentina  2000-11-01   17.337
152  Argentina  2000-10-01   15.835
153  Argentina  2000-09-01   11.663
154  Argentina  2000-08-01    9.887
155  Argentina  2000-07-01    6.306
156  Argentina  2000-06-01    8.794
157  Argentina  2000-05-01   10.858
158  Argentina  2000-04-01   15.438
159  Argentina  2000-03-01   17.929
160  Argentina  2000-02-01   20.395

In [28]:
fig=px.sunburst(df_some_countries,path=['Country','date','AvgTemp'],color='AvgTemp',hover_name='Country')
fig.show()

In [29]:
plotly.offline.plot(fig, filename='using_sunburst_plotly.express.html')

'using_sunburst_plotly.express.html'